# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [63]:
df = pd.read_csv('/datasets/insurance.csv')

In [64]:
display(df.info())
display(df.head(10))
display(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


(5000, 5)

### Вывод:

Пропусков нет, данные записанны в верном типе

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится

**Обоснование:** $$
a = X_1w
$$
$
a = X_1(X^TX)^{-1}X^Ty=X_1X^{-1}(X^T)^{-1}X^Ty=X_1X^{-1}y
$
$
a_1 = (X_1A)((XA)^TXA)^{-1}(XA)^Ty=(X_1A)(XA)^{-1}((XA)^T)^{-1}(XA)^Ty=(X_1A)A^{-1}X^{-1}(X^T)^{-1}(A^T)^{-1}A^TX^Ty=X_1AA^{-1}X^{-1}(X^T)^{-1}(A^T)^{-1}A^TX^Ty=X_1X^{-1}y
$

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$

$$
w' = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$
$$
w' = P^{-1}(X^T X)^{-1} X^T y
$$

## Алгоритм преобразования

**Алгоритм**

Преобразуем исходные данные с помощью умножения на рандомную матрицу и проверим показатель R2

**Обоснование**

1. Ищем метрики качества на исходных данных
2. Создаём рандомную матрицу и проверяем её на обратимость
3. Ищем метрики качества на преобразованных данных
4. Сравниваем результаты


## Проверка алгоритма

In [65]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=123)

In [66]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("R2 =", r2_score(target_test, predictions))

R2 = 0.43018469990932917


In [67]:
# Функция для создания рандомной матрицы
def generate_invertible_matrix(size):
    try:
        matrix = np.random.normal(size=(size, size))
        # проверим матрицу на обратимость, если нет, пробуем сгенерировать еще раз
        # таким образом гарантируем, что матрица стопроцентно будет обратимой
        np.linalg.inv(matrix)
    except np.linalg.LinAlgError:
        matrix = generate_invertible_matrix()
    
    return matrix

In [68]:
# создадим рандомную матрицу
random_matrix = generate_invertible_matrix(4)
display(random_matrix)
random_matrix.shape

array([[-0.23306492, -1.82688085, -0.25079012, -0.84715159],
       [-0.35777034, -0.47156139,  0.13550397, -0.46403923],
       [ 0.39164912,  0.14727591, -0.32701937, -0.35219358],
       [ 0.76806697,  0.88269302, -0.37557001,  0.10709493]])

(4, 4)

Функцию использовал для создания рандомной матрицы выше. Спасибо за помощь

In [69]:
# перемножим тестовые и тренировочную выборку на рандомную матрицу
features_train_new = features_train @ random_matrix
features_test_new = features_test @ random_matrix
display(features_train_new.shape)
display(features_test_new.shape)
display(features_test_new.head(5))
features_train_new.head(5)

(3750, 4)

(1250, 4)

,0,1,2,3
2648,15381.201916,5772.441000,-12848.182612,-13855.511803
2456,14793.422996,5550.995147,-12357.622608,-13328.506463
4557,21137.222969,7935.268874,-17655.243657,-19037.897453
4884,15732.130335,5904.458390,-13141.571740,-14173.959065
92,10448.622183,3919.533505,-8728.656073,-9415.909501


,0,1,2,3
2413,16674.822368,6259.745319,-13927.962186,-15018.000451
1471,9741.864920,3652.079065,-8139.343678,-8784.281261
1196,17850.613274,6704.463904,-14908.831404,-16071.163980
1509,13808.451171,5176.615716,-11537.519813,-12455.064039
4110,14636.710823,5492.145214,-12226.710296,-13186.808049


In [70]:
model = LinearRegression()
model.fit(features_train_new, target_train)
predictions = model.predict(features_test_new)
print("R2 =", r2_score(target_test, predictions))

R2 = 0.4301846999089898


### Вывод:

После умножения данных на рандомную матрицу, показатель R2 не поменялся.

## Общий вывод:

Разработал метод преобразования данных, путем умножения на рандомную матрицу. Данные получились "не читаемы", показатель качества R2 показал такой же результат как и на не измененных данных.